# Find UniProt IDs in ChEMBL targets

Ultimately, we are interested in getting [activity data from ChEMBl](/chembl-27/query_local_chembl-27.ipynb) we need to account for three components:

* The compound being measured
* The target the compound binds to
* The assay where this measurement took place

So, to find all activity data stored in ChEMBL that refers to kinases, we have to query for those assays annotated with a certain target.

Each of those three components have a unique ChEMBL ID, but so far we only have obtained Uniprot IDs in the `human-kinases` notebook. We need a way to connect Uniprot IDs to ChEMBL target IDs. Fortunately, ChEMBL maintains such a map in their FTP releases. We will parse that file and convert it into a dataframe for easy manipulation.

In [1]:
from pathlib import Path
import urllib.request

import chembl_downloader
import pandas as pd

In [2]:
REPO = (Path(_dh[-1]) / "..").resolve()
DATA = REPO / 'data'

In [3]:
# CHEMBL_VERSION = chembl_downloader.latest()
CHEMBL_VERSION = 30

In [4]:
uniprot_map = chembl_downloader.get_uniprot_mapping_df(version=CHEMBL_VERSION)
uniprot_map = uniprot_map.rename(columns={
    "uniprot_id": "UniprotID",
    "chembl_target_id": "chembl_targets", 
    "name": "description", 
})
uniprot_map

,UniprotID,chembl_targets,description,type
0,P21266,CHEMBL2242,Glutathione S-transferase Mu 3,SINGLE PROTEIN
1,O00519,CHEMBL2243,Anandamide amidohydrolase,SINGLE PROTEIN
2,P19217,CHEMBL2244,Estrogen sulfotransferase,SINGLE PROTEIN
3,P97292,CHEMBL2245,Histamine H2 receptor,SINGLE PROTEIN
4,P17342,CHEMBL2247,Atrial natriuretic peptide receptor C,SINGLE PROTEIN
...,...,...,...,...
13526,A0A0J5PZ55,CHEMBL4630893,"Dihydroorotate dehydrogenase (quinone), mitoch...",SINGLE PROTEIN
13527,Q8NI17,CHEMBL4630894,Interleukin-31 receptor subunit alpha,SINGLE PROTEIN
13528,P43630,CHEMBL4630895,Killer cell immunoglobulin-like receptor 3DL2,SINGLE PROTEIN
13529,A0A0H3HP34,CHEMBL4630896,Enoyl-[acyl-carrier-protein] reductase [NADH],SINGLE PROTEIN


We join this new information to the human kinases aggregated list from `human-kinases` (all of them, regardless the source):

In [5]:
kinases = pd.read_csv(DATA / "human_kinases.aggregated.csv", index_col=0)
kinases

,UniprotID,Name,kinhub,klifs,pkinfam,reviewed_uniprot,dunbrack_msa
0,A0A0B4J2F2,SIK1B,False,False,True,True,True
1,A4QPH2,PI4KAP2,False,True,False,False,False
2,B5MCJ9,TRIM66,True,False,False,False,False
3,O00141,SGK1,True,True,True,True,True
4,O00238,BMPR1B|BMR1B,True,True,True,True,True
...,...,...,...,...,...,...,...
547,Q9Y616,IRAK3,True,True,True,True,True
548,Q9Y6E0,STK24,True,True,True,True,True
549,Q9Y6M4,CSNK1G3|KC1G3,True,True,True,True,True
550,Q9Y6R4,M3K4|MAP3K4,True,True,True,True,True


We are only interested in those kinases present in these datasets:

* KinHub
* KLIFS
* PKinFam
* Dunbrack's MSA

In [6]:
kinases_subset = kinases[kinases[["kinhub", "klifs", "pkinfam", "dunbrack_msa"]].sum(axis=1) > 0]
kinases_subset

,UniprotID,Name,kinhub,klifs,pkinfam,reviewed_uniprot,dunbrack_msa
0,A0A0B4J2F2,SIK1B,False,False,True,True,True
1,A4QPH2,PI4KAP2,False,True,False,False,False
2,B5MCJ9,TRIM66,True,False,False,False,False
3,O00141,SGK1,True,True,True,True,True
4,O00238,BMPR1B|BMR1B,True,True,True,True,True
...,...,...,...,...,...,...,...
547,Q9Y616,IRAK3,True,True,True,True,True
548,Q9Y6E0,STK24,True,True,True,True,True
549,Q9Y6M4,CSNK1G3|KC1G3,True,True,True,True,True
550,Q9Y6R4,M3K4|MAP3K4,True,True,True,True,True


We would also like to preserve the provenance of the Uniprot assignment, so we will group the provenance columns in a single one now.

In [7]:
kinases_subset["origin"] = kinases_subset.apply(lambda s: '|'.join([k for k in ["kinhub", "klifs", "pkinfam", "reviewed_uniprot", "dunbrack_msa"] if s[k]]), axis=1)
kinases_subset

,UniprotID,Name,kinhub,klifs,pkinfam,reviewed_uniprot,dunbrack_msa,origin
0,A0A0B4J2F2,SIK1B,False,False,True,True,True,pkinfam|reviewed_uniprot|dunbrack_msa
1,A4QPH2,PI4KAP2,False,True,False,False,False,klifs
2,B5MCJ9,TRIM66,True,False,False,False,False,kinhub
3,O00141,SGK1,True,True,True,True,True,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
4,O00238,BMPR1B|BMR1B,True,True,True,True,True,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
...,...,...,...,...,...,...,...,...
547,Q9Y616,IRAK3,True,True,True,True,True,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
548,Q9Y6E0,STK24,True,True,True,True,True,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
549,Q9Y6M4,CSNK1G3|KC1G3,True,True,True,True,True,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
550,Q9Y6R4,M3K4|MAP3K4,True,True,True,True,True,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...


We can now merge the needed columns based on the `UniprotID` key.

In [8]:
merged = pd.merge(kinases_subset[["UniprotID", "Name", "origin"]], uniprot_map[["UniprotID", "chembl_targets", "type"]], how="inner", on='UniprotID')[["UniprotID", "Name", "chembl_targets", "type", "origin"]]
merged

,UniprotID,Name,chembl_targets,type,origin
0,A4QPH2,PI4KAP2,CHEMBL4105789,SINGLE PROTEIN,klifs
1,A4QPH2,PI4KAP2,CHEMBL3038509,PROTEIN COMPLEX,klifs
2,O00141,SGK1,CHEMBL2343,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
3,O00238,BMPR1B|BMR1B,CHEMBL5476,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
4,O00311,CDC7,CHEMBL2111377,PROTEIN COMPLEX,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
...,...,...,...,...,...
1014,Q9Y616,IRAK3,CHEMBL4748234,PROTEIN-PROTEIN INTERACTION,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
1015,Q9Y616,IRAK3,CHEMBL4742326,PROTEIN-PROTEIN INTERACTION,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
1016,Q9Y6E0,STK24,CHEMBL5082,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
1017,Q9Y6M4,CSNK1G3|KC1G3,CHEMBL5084,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...


~~How is this possible? 969 targets (ChEMBL 28)?!~~

Apparently, there's not 1:1 correspondence between UniprotID and ChEMBL ID! Some Uniprot IDs are included in several ChEMBL targets:

In [9]:
merged.UniprotID.value_counts()

P11802    17
P24941    15
P35968    11
Q00534    11
P06493    11
          ..
O75962     1
Q59H18     1
Q5JZY3     1
Q5MAI5     1
Q9Y6R4     1
Name: UniprotID, Length: 496, dtype: int64

In [10]:
merged[merged.UniprotID == "P11802"]

,UniprotID,Name,chembl_targets,type,origin
221,P11802,CDK4,CHEMBL331,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
222,P11802,CDK4,CHEMBL2095942,PROTEIN COMPLEX,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
223,P11802,CDK4,CHEMBL2111326,SELECTIVITY GROUP,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
224,P11802,CDK4,CHEMBL1907601,PROTEIN COMPLEX,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
225,P11802,CDK4,CHEMBL3301385,PROTEIN COMPLEX,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
226,P11802,CDK4,CHEMBL4523686,PROTEIN-PROTEIN INTERACTION,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
227,P11802,CDK4,CHEMBL4523715,PROTEIN-PROTEIN INTERACTION,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
228,P11802,CDK4,CHEMBL4523732,PROTEIN-PROTEIN INTERACTION,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
229,P11802,CDK4,CHEMBL3038472,PROTEIN COMPLEX,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
230,P11802,CDK4,CHEMBL4523963,SELECTIVITY GROUP,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...


... and some ChEMBL targets include several kinases (e.g. chimeric proteins):

In [11]:
merged[merged.chembl_targets == "CHEMBL2096618"]

,UniprotID,Name,chembl_targets,type,origin
113,P00519,ABL1,CHEMBL2096618,CHIMERIC PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
209,P11274,BCR,CHEMBL2096618,CHIMERIC PROTEIN,kinhub|klifs


This is due to the different `type` values:

In [12]:
merged.type.value_counts()

SINGLE PROTEIN                 494
PROTEIN FAMILY                 247
PROTEIN COMPLEX                121
PROTEIN-PROTEIN INTERACTION    108
SELECTIVITY GROUP               22
CHIMERIC PROTEIN                16
PROTEIN COMPLEX GROUP           11
Name: type, dtype: int64

If we focus on `SINGLE PROTEIN` types:

In [13]:
merged[merged.type == "SINGLE PROTEIN"]

,UniprotID,Name,chembl_targets,type,origin
0,A4QPH2,PI4KAP2,CHEMBL4105789,SINGLE PROTEIN,klifs
2,O00141,SGK1,CHEMBL2343,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
3,O00238,BMPR1B|BMR1B,CHEMBL5476,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
5,O00311,CDC7,CHEMBL5443,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
7,O00329,PIK3CD,CHEMBL3130,SINGLE PROTEIN,klifs|pkinfam
...,...,...,...,...,...
1012,Q9Y5S2,MRCKB|CDC42BPB,CHEMBL5052,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
1013,Q9Y616,IRAK3,CHEMBL5081,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
1016,Q9Y6E0,STK24,CHEMBL5082,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...
1017,Q9Y6M4,CSNK1G3|KC1G3,CHEMBL5084,SINGLE PROTEIN,kinhub|klifs|pkinfam|reviewed_uniprot|dunbrack...


... we end up with a total of 491 targets (ChEMBL 28), which is more acceptable.

For that reason, we will only save records corresponding to `type == SINGLE PROTEIN`

In [14]:
merged[merged.type == "SINGLE PROTEIN"].to_csv(DATA /  f"human_kinases_and_chembl_targets.chembl_{CHEMBL_VERSION}.csv", index=False)